# **Basic TwoCrypto-NG Contract Methods**

**WARNING:** These example methods are for fork mode only. When calling these functions in production, make sure to set the parameters accordingly. For simplicity, we set the minimum amounts for adding/removing liquidity and exchanging tokens to 0. DO NOT do this in production, as you WILL suffer losses!

*Not covered (will have their own notebooks):*  
- `exchange_extended`
- Oracles and their behavior
- Admin controls such as changing parameters


In [1]:
import boa

In [2]:
admin = boa.env.generate_address()
fee_receiver = boa.env.generate_address()
trader = boa.env.generate_address()
print(admin, fee_receiver, trader)

0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E 0xA73d7cddCf77c00827459f986bf828999B58C6Fe 0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B


## **Coin mocks**

Creating two mock coins (`coin_a`, `coin_b`) and mint some tokens to `trader`.

In [3]:
coin_a = boa.load("../../contracts/twocrypto-ng/mocks/ERC20Mock.vy", "coin_a", "coin_a", 18)
coin_b = boa.load("../../contracts/twocrypto-ng/mocks/ERC20Mock.vy", "coin_b", "coin_b", 18)

coin_a._mint_for_testing(trader, 10**25, sender=trader)
coin_b._mint_for_testing(trader, 10**25, sender=trader)
print(coin_a, coin_b)

/Users/mo/Documents/CurveDocs/venv-notebook/lib/python3.11/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")


<../../contracts/twocrypto-ng/mocks/ERC20Mock.vy at 0x0880cf17Bd263d3d3a5c09D2D86cCecA3CcbD97c, compiled with vyper-0.3.10+9136169>
<storage: name=coin_a                          , symbol=coin_a                          , decimals=18, balanceOf={'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': 10000000000000000000000000}, allowances={}, totalSupply=10000000000000000000000000> <../../contracts/twocrypto-ng/mocks/ERC20Mock.vy at 0x2cb6bCe32aeF4eD506382896e702DE7Ff109D9E9, compiled with vyper-0.3.10+9136169>
<storage: name=coin_b                          , symbol=coin_b                          , decimals=18, balanceOf={'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': 10000000000000000000000000}, allowances={}, totalSupply=10000000000000000000000000>


## **Deploy TwoCrypto-NG Infrastructure**

First, we need to deploy the TwoCrypto-NG infrastructure, consisting of a pool implementation, math and views contract, and the pool factory. For simplicity, we have generated some **random addresses** for `admin` and `fee_receiver`.

In [4]:
pool_implementation = boa.load_partial("../../contracts/twocrypto-ng/main/CurveTwocryptoOptimized.vy")
gauge_implementation = boa.load_partial("../../contracts/twocrypto-ng/main/LiquidityGauge.vy")
math_implementation = boa.load("../../contracts/twocrypto-ng/main/CurveCryptoMathOptimized2.vy")
views_implementation = boa.load("../../contracts/twocrypto-ng/main/CurveCryptoViews2Optimized.vy")

with boa.env.prank(admin):
    factory = boa.load("../../contracts/twocrypto-ng/main/CurveTwocryptoFactory.vy")
    factory.initialise_ownership(fee_receiver, admin)

    factory.set_pool_implementation(pool_implementation.deploy_as_blueprint().address, 0)
    factory.set_gauge_implementation(gauge_implementation.deploy_as_blueprint().address)
    factory.set_math_implementation(math_implementation.address)
    factory.set_views_implementation(views_implementation.address)

print(
    "Factory: " + factory.address + "\n" +
    "Pool Implementation: " + factory.pool_implementations(0) + "\n" +
    "Gauge Implementation: " + factory.gauge_implementation() + "\n" +
    "Math Implementation: " + factory.math_implementation() + "\n" +
    "Views Implementation: " + factory.views_implementation()
    )

/Users/mo/Documents/CurveDocs/venv-notebook/lib/python3.11/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


Factory: 0xC6Acb7D16D51f72eAA659668F30A40d87E2E0551
Pool Implementation: 0x3d06E92f20305D9a2D71a1D479E9EE22690Ae7E4
Gauge Implementation: 0x78548820b365886d05009F1127bf553603E5A836
Math Implementation: 0xB822167C7EefF0B53DcfDEE2D8fe73dEDB25505b
Views Implementation: 0xDa5e407C7b1887E7f76c920B70614e73feA0DDA1


## **Deploying a Liquidity Pool**

We can now deploy a liquidity pool from the factory contract. Therefore, we set some values for the parameters. 

**NOTE:** Some of these parameters may have upper or lower bounds. When deploying, please make sure to use valid values.

In [5]:
# prepare pool params for deployment

A = 400000
gamma = 145000000000000
mid_fee = 26000000
out_fee = 45000000
allowed_extra_profit = 2000000000000
fee_gamma = 230000000000000
adjustment_step = 146000000000000
ma_time = 600
xcp_ma_time = 1800 * 24
initial_price = 10**18  # 1:1 at the start
admin_fee = 5 * 10**9

In [6]:
# deploying the pool

pool = factory.deploy_pool(
    "ng", "ng", [coin_a.address, coin_b.address],
    0, A, gamma, mid_fee, out_fee, fee_gamma, allowed_extra_profit, adjustment_step, int(ma_time/0.693), initial_price
)
pool = pool_implementation.at(pool)
pool

<../../contracts/twocrypto-ng/main/CurveTwocryptoOptimized.vy at 0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2, compiled with vyper-0.3.10+9136169>
<storage: cached_price_scale=1000000000000000000, cached_price_oracle=1000000000000000000, cached_xcp_oracle=0, last_prices=1000000000000000000, last_timestamp=580275158891956380444168496012540517589130178596, last_xcp=0, xcp_ma_time=62324, initial_A_gamma=136112946768375385385349842972852284582400000, initial_A_gamma_time=0, future_A_gamma=136112946768375385385349842972852284582400000, future_A_gamma_time=0, balances=[0, 0], D=0, xcp_profit=0, xcp_profit_a=1000000000000000000, virtual_price=0, packed_rebalancing_params=680564733841876929619973849625130958848000000000865, packed_fee_params=8847341539944400050877843276543133320576000000, last_admin_fee_claim_timestamp=0, admin_lp_virtual_balance=0, balanceOf={}, allowance={}, totalSupply=0, nonces={}>

## **Query Basic Variables of the Pool**

We can now query variables from the contract to check if the pool has been deployed correctly.


In [7]:
print(
    "A: " + str(pool.A()) + "\n" +
    "gamma: " + str(pool.gamma()) + "\n" +
    "mid_fee: " + str(pool.mid_fee()) + "\n" +
    "out_fee: " + str(pool.out_fee()) + "\n" +
    "allowed_extra_profit: " + str(pool.allowed_extra_profit()) + "\n" +
    "fee_gamma: " + str(pool.fee_gamma()) + "\n" +
    "adjustment_step: " + str(pool.adjustment_step()) + "\n" +
    "ma_time: " + str(pool.ma_time()) + "\n" +
    "xcp_ma_time: " + str(pool.xcp_ma_time()) + "\n" +
    "ADMIN_FEE: " + str(pool.ADMIN_FEE()) + "\n" +
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader)/10**18) + "\n" +
    "trader balance coin_a: " + str(coin_b.balanceOf(trader)/10**18)
)


A: 400000
gamma: 145000000000000
mid_fee: 26000000
out_fee: 45000000
allowed_extra_profit: 2000000000000
fee_gamma: 230000000000000
adjustment_step: 146000000000000
ma_time: 600
xcp_ma_time: 62324
ADMIN_FEE: 5000000000
pool balance coin_a: 0
pool balance coin_b: 0
trader balance coin_a: 10000000.0
trader balance coin_a: 10000000.0


## **Add and Remove Liquidity**

We have successfully created a TwoCrypto-NG liquidity pool with some valid parameters. 

Before we can exchange tokens, we need to ensure there is liquidity in the pool. However, even before that, we need to approve the contract to be able to spend our coins (`coin_a`, `coin_b`).

In [8]:
# grant max approval

with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        coin.approve(pool, 2**256-1)

print(coin_a._storage.allowances.get())
print(coin_b._storage.allowances.get())

{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}
{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}


In [9]:
# We now add liquidity in a balanced portion.

with boa.env.prank(trader):
    pool.add_liquidity([10**20, 10**20], 0)

print(
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader lp token balance: " + str(pool.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_b: " + str(coin_b.balanceOf(trader) / 10**18)
)

pool balance coin_a: 100000000000000000000
pool balance coin_b: 100000000000000000000
trader lp token balance: 100.0
trader balance coin_a: 9999900.0
trader balance coin_b: 9999900.0


In [10]:
# We also add liquidity in unbalanced portions or even fully single-sided.

with boa.env.prank(trader):
    pool.add_liquidity([10**19, 0], 0)

print(
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader lp token balance: " + str(pool.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_b: " + str(coin_b.balanceOf(trader) / 10**18)
)

pool balance coin_a: 110000000000000000000
pool balance coin_b: 100000000000000000000
trader lp token balance: 104.9103149267059
trader balance coin_a: 9999890.0
trader balance coin_b: 9999900.0


In [11]:
# remove liquidity via `remove_liquidity()`; this removes liquidity in a balanced portion

with boa.env.prank(trader):
    pool.remove_liquidity(2**18, [0, 0])

print(
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader lp token balance: " + str(pool.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_b: " + str(coin_b.balanceOf(trader) / 10**18)
)

pool balance coin_a: 109999999999999725140
pool balance coin_b: 99999999999999750127
trader lp token balance: 104.91031492670564
trader balance coin_a: 9999890.0
trader balance coin_b: 9999900.0


In [12]:
# remove liquidity one sided

with boa.env.prank(trader):
    pool.remove_liquidity_one_coin(10**18, 0, 0)

print(
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader lp token balance:" + str(pool.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_b: " + str(coin_b.balanceOf(trader) / 10**18)
)

pool balance coin_a: 107925361718425750945
pool balance coin_b: 99992357851252493594
trader lp token balance:103.91031492670564
trader balance coin_a: 9999892.066231918
trader balance coin_b: 9999900.0


# **Exchanging Tokens**

Now that the pool has some liquidity, we can do a token exchange. Lets swap 10^18 (`dx`) coins of coin(1) (`i`) for coin(0) (`j`).

NOTE: `exchange_extended` will be showcased in a seperate notebook!

In [13]:
before_a = coin_a.balanceOf(trader)
before_b = coin_b.balanceOf(trader)

with boa.env.prank(trader):
    pool.exchange(1, 0, 10**18, 0)

after_a = coin_a.balanceOf(trader)
after_b = coin_b.balanceOf(trader)

boa.env.time_travel(100)

print(
    "pool balance coin_a: " + str(pool.balances(0)) + "\n" +
    "pool balance coin_b: " + str(pool.balances(1)) + "\n" +
    "trader balance coin_a: " + str(coin_a.balanceOf(trader) / 10**18) + "\n" +
    "trader balance coin_b: " + str(coin_b.balanceOf(trader) / 10**18) + "\n" +
    "diff coin_a: " + str((after_a - before_a) / 10**18) + "\n" +
    "diff coin_b: " + str((after_b - before_b) / 10**18)
)

pool balance coin_a: 106879520235153785405
pool balance coin_b: 100992357851252493594
trader balance coin_a: 9999893.112073401
trader balance coin_b: 9999899.0
diff coin_a: 1.0458414832719656
diff coin_b: -1.0
